In [ ]:
from functions_needed import *

In [5]:
%%time
#import segmind.instance as instance
import requests
import traceback

try:
    # Lipkin Model Cross-Section
    import numpy as np
    from scipy.integrate import odeint, odepack
    import matplotlib.pyplot as plt
    from multiprocessing import Pool
    from scipy.special import j0, jn_zeros
    import warnings, time
    from functions_needed import *
    warnings.filterwarnings("ignore")
    from qutip import Options
    from qutip.wigner import qfunc
    import h5py, tqdm
    from functions_needed import *

    # Maximum number of strobes
    nmax = 1000
    # Number of Processors
    nprocs = 8

    def lmg(z, t, h0, h, w):
        q, p = z
        drive = h0 + h * np.cos(w * t)
        sterm = np.sqrt(1.0 - 4.0 * q * q)
        qdot = drive * sterm * np.sin(p)
        pdot = 4.0 * q * (1.0 - (drive * np.cos(p)/sterm) )
        return [qdot, pdot]

    def poincare_sec(x0, p0, nmax, args):
        h0, h, omega = args
        T = 2.0 * np.pi/omega
        t = T * np.arange(nmax)
        init = [x0,p0]
        points = odeint(lmg, init, t, args=args)
        points[:,1] = points[:,1] % (2.0 * np.pi)
        return points

    # Initial Conditions for classical poincare
    n_ic = 20
    init_x = np.linspace(-0.45, 0.45, n_ic)
    near_zero = np.linspace(-0.01, 0.01, 10)
    init_x = np.sort(np.append(init_x, near_zero))
    init_p = np.linspace(0.0, 2.0 * np.pi, n_ic)
    init_p = np.sort(np.append(init_p, [0.1, np.pi + 0.2]))
    xv, pv = np.meshgrid(init_x, init_p, indexing='ij')

    # Initial Conditions for quantum husimi
    # Choose a size
    N = 500
    #Qutip Solver Options
    opts = Options(nsteps=1e5, num_cpus=1, openmp_threads=1)
    n_Q = 100
    Q_qvals = np.linspace(-0.5, 0.5, n_Q)
    Q_pvals = np.linspace(0.0, 2.0 * np.pi, n_Q)
    Q_pvals = np.sort(np.append(Q_pvals, [0.1, np.pi + 0.2]))
    # Drive parameters
    h0 = np.sqrt(3)
    omegas = [2.5, 90.0]
    eta = jn_zeros(0,5)
    eta = eta[0]

    for i, w in enumerate(omegas):
        h_frz = eta * w/4
        print(f"Poincare section dynamics for h = {h_frz} w = {w} with {nprocs} processes ...")   
        start = time.time()
        with Pool(processes = nprocs) as p:
            poincare_data = p.starmap(poincare_sec,[(x0, p0, nmax, (h0, h_frz, w)) for x0, p0 in zip(xv.flatten(), pv.flatten())])

        poincare_data = np.vstack(poincare_data)
        qvals = poincare_data[:,0]
        pvals = poincare_data[:,1]        

        #Now for the quantum case
        print(f"Husimi plots for h = {h_frz} w = {w} with {nprocs} processes ...")  
        start = time.time()
        params = {'h0':0, 'h':h_frz, 'omega':w, 'N':N, 'opts':opts} 
        H0, H1 = get_hamiltonians(N)
        H = [H0,[H1,drive]]
        T = 2 * np.pi/w
        evecs_f, _ = floquet_modes_mod(H, T, args=params)
        Q_fstate = 0
        for fstate in evecs_f:
            Q_fstate += qfunc(fstate, Q_qvals, Q_pvals)

        filename = 'classical_lipkin_qpfstate_N'+str(N)+'_w'+ str(w)+'_.hdf5'
        with h5py.File(filename, 'w') as hf:
            hf.create_dataset('Q_fstate', np.shape(Q_fstate), data= Q_fstate)
            hf.create_dataset('pvals', np.shape(pvals), data= pvals)
            hf.create_dataset('qvals', np.shape(qvals), data= qvals)
            hf.create_dataset('omegas', np.shape(omegas), data= omegas)
            hf.create_dataset('Q_pvals', np.shape(Q_pvals), data= Q_pvals)
            hf.create_dataset('Q_qvals', np.shape(Q_qvals), data= Q_qvals)
            hf.attrs['N'] = N
            hf.attrs['h_frz'] = h_frz
            hf.close()
    print("run complete!")
    bot_message = "Code executed with no exceptions.\n"
except Exception:
    # This bit exists in case your code conks out with an exception
    # That way, the exception will no longer terminate your execution
    # without stopping the segmind instance
    print(traceback.format_exc())
    bot_message = f"Code exited with {Exception}. Check stderr for details.\n"

#####################################################
# You can customize the bot message as per your needs
#####################################################
whoami = "Analabha Roy"
# datetime object containing current date and time
now = datetime.now()
bot_message = 'Program run by ' + whoami + ' has been completed. Date/Time of termination:' + str(now) + bot_message
#####################################################

# The SegmindStatusBot is sent an instruction to broadcast the message in 'bot_message'
bot_token = 'PUT BOT TOKEN'
bot_chatID = 'PUT CHAT ID'
send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
#response = requests.get(send_text)

from ipylab import JupyterFrontEnd
app = JupyterFrontEnd()

app.commands.execute('docmanager:save')

# The running instance is now automatically stopped so that it doesn't eat any more credits than it has to
#instance.stop()
print("Done!")

Definitions complete!
Running for N =  10 with 4 processors


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 103/103 [00:00<00:00, 32346.01it/s]



Done. Now calculating PR ...
Exact simulation complete! Saving Data ...
Done!
CPU times: user 29.3 ms, sys: 14.2 ms, total: 43.5 ms
Wall time: 12.8 s


In [ ]:
!ls classical_lipkin_qpfstate*>filedata_classical_lipkin_qpfvals.txt

In [ ]:
import matplotlib.pyplot as plt
import h5py
from scipy.special import j0,jn_zeros
import numpy as np

plt.rcParams.update({
    "figure.figsize": (10, 10),
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.size": 10,
    "font.sans-serif": ["Computer Modern"]})

fig, axs = plt.subplots(2, 2)

with open("filedata_classical_lipkin_qpfvals.txt", 'r') as fp:
    lines = fp.readlines()
    x = len(lines)

    for i in range(0,x):
        filename = lines[i][:-1]
        print(filename)
        with h5py.File(filename, 'r') as f: 
            print('check everytime if keys has correct order!!\n',list(f.keys()))
            Q_fstate = f[list(f.keys())[0]][()] 
            Q_pvals = f[list(f.keys())[1]][()] 
            Q_qvals = f[list(f.keys())[2]][()]
            omegas = f[list(f.keys())[3]][()]
            pvals = f[list(f.keys())[4]][()]
            qvals = f[list(f.keys())[5]][()]
            N = f.attrs['N']
            h_frz = f.attrs['h_frz']

        w = omegas[i]
        print(w)
        axs[0,i].set_title(r'$N={:d}$ (Husimi), $\omega = {:2.1f}, J_0(4 h/\omega) = {:1.3f}$'.format(N,w,j0(4*h_frz/w)))
        axs[0,i].scatter(qvals, pvals , color='blue', marker='.', s=.1)  
        #axs[1,i].contourf(Q_qvals, Q_pvals, Q_fstate/(2*N+1), 500)
        axs[1,i].contourf(Q_qvals, Q_pvals, Q_fstate, 200, cmap='inferno')
        axs[1,i].set_xlabel(r'$q$')
        
#Formatting
x_tick = np.array([-0.5, 0.0, 0.5])
unit   = 1.0
p_tick = np.arange(0.0, 2.0 + unit, unit)
p_label = [r"$0$", r"$\pi$", r"$2\pi$"]

for ax in axs.flatten():
    ax.set_xticks(x_tick)
    ax.set_xlim(-0.52, 0.52)
    ax.set_yticks(p_tick*np.pi, p_label)
    ax.set_ylabel(r"$p$")
    ax.tick_params(direction='in', length=8, width=3)

plt.savefig("lmg_poincare01.jpeg", dpi = 300)
plt.show()